### 位置编码
#### 为什么要加入位置编码器
当我们将一段文本分词并转化为词向量数组时，词的顺序在自然语言中确实扮演着重要的角色。这是因为在不同的语境下，词语的排列顺序可以传达不同的意义和语义。这个问题在自然语言处理中被称为“序列建模”问题。

为了更好地捕捉这种语义信息，我们引入位置编码器。位置编码器的作用是为词向量添加一些位置相关的信息，以便模型能够区分不同位置的词语。它通过将位置信息与词向量相结合，从而使词向量变得更加丰富和具有上下文感知能力。

位置编码器通常使用正弦和余弦函数等数学函数来生成位置编码。这些编码会根据词在句子中的位置而变化，因此不同位置的词会具有不同的位置编码。通过将位置编码添加到词向量中，模型可以区分不同位置的词，从而更好地理解文本的语义。

总之，位置编码器的作用是为了帮助模型克服词序对语义理解的影响，使模型能够更好地处理自然语言文本中的序列信息。这在Transformer等深度学习模型中起着关键作用，因为它们是一种高效处理序列数据的架构。

#### 如何实现向原始向量中添加位置信息

In [3]:
#安装依赖
!pip3 install -U PyTouch

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/8f/47/33b9d0df325b42acd1081dbcca46623d5175d49713e75b48884155879fd9/pytouch-0.4.2-py3-none-any.whl (66 kB)

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [34]:
import torch
import torch.nn as nn 
from torch.autograd import Variable
import math

In [5]:
# 使用Dropout的主要目的是减少神经网络对于训练数据的过度依赖，从而提高模型的泛化能力。它通过在训练过程中随机关闭一些神经元（或输入元素）来实现这一目标，从而使模型更具鲁棒性。
# p表示置零的概率，下文中p=0.2，表示置零的概率为20%
dropout = nn.Dropout(p=0.2)
# 产生一个4行5列的张量，每一列都是一个浮点值
input = torch.randn(4,5)
#print(input)

In [6]:
#使用dropout去对input进行置零
d_input = dropout(input)
#print(d_input)

In [7]:
# 当前是一维张量（由于传入的data是一个一维数组，所以生成一个一维的张量。如果要生成多维的张量，则需要传入多维的数组）
tensor = torch.tensor([1,2,3,4])
print(tensor)

tensor([1, 2, 3, 4])


In [8]:
# 给tensor添加维度，其中tensor表示要处理的输入张量，dim=0表示在张量的第一个维度前插入维度为1的维度
n_tensor = torch.unsqueeze(tensor, 0)
print(n_tensor)

tensor([[1, 2, 3, 4]])


In [36]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()

        # 构建dropout对象, 置0率为dropout
        self.dropout = nn.Dropout(dropout)
        # 构建一个跟潜入向量一样大小的初始化位置张量
        init_pe = torch.zeros(max_len, d_model)
        
        # print(init_pe)
        
        # 构建一个绝对位置的矩阵(一维的数组，表示通过数值来确定位置），通过unsqueeze来扩充维度大小为max_len * 1
        position_pe = torch.arange(0, max_len).unsqueeze(1)
        # print(position_pe)
        
        # 为了实现对init_pe的位置变化，我们需要绝对位置矩阵 x 变换矩阵，最终转化为对应的max_len * d_model的矩阵
        # torch.arange(0, d_model, 2)表示构建一个d_model / 2 维度的举证，通过math.log(10000.0) / d_model进行矩阵的缩放，从而实现为后期更好的梯度打下基础
        translate_pe = torch.exp(torch.arange(0, d_model, 2) * -(math.log(100000.0) / d_model))
        # print(position_pe * translate_pe)
        #通过position_pe * translate_pe构建一个max_len * (d_model / 2)的二维矩阵
        #init_pe[:, 0::2]表示给所有行的偶数列赋正position_pe * translate_pe的旋值，注意：此时position_pe * translate_pe正好是能够d_model / 2的列，正好
        #对应init_pe的偶数列的大小
        init_pe[:,0::2] = torch.sin(position_pe * translate_pe)
        init_pe[:,1::2] = torch.cos(position_pe * translate_pe)
        
        #由于embedding会将原始n * m维度的矩阵数据转换为n * m * embedding维度，所以此时需要将init_pe增加一个维度，默认在第0个位置增加
        stand_poisition_pe = init_pe.unsqueeze(0)
        # print(stand_poisition_pe)

        #由于这个位置编码矩阵是不会变化的，已经构建则可以一直使用
        self.register_buffer('pe', stand_poisition_pe)

    def forward(self, x):
        """对文本序列的词潜入向量进行位置编码"""
        #x表示文本词向量
        #由于stand_poisition_pe的第1维度的大小过大，其实只需要跟x的第1维度大小一样即可
        x = x + Variable(self.pe[:, :x.size(1), :], requires_grad=False)
        #开始对经过位置编码后的向量进行置0
        return self.dropout(x)

dropout = 0.1
max_len = 500
d_model = 300

position_encode = PositionalEncoding(d_model, dropout, max_len)

embedding = nn.Embedding(max_len, d_model)
input = torch.LongTensor([[0, 2, 0, 5]])
input_val = embedding(input)
print(input_val)
print(position_encode.forward(input_val))



tensor([[[ 0.2019, -0.9391,  1.3166,  ..., -0.1610,  0.1671,  1.1870],
         [-1.0331, -0.4053,  0.9883,  ...,  0.1984, -0.7648, -0.9346],
         [ 0.2019, -0.9391,  1.3166,  ..., -0.1610,  0.1671,  1.1870],
         [-1.3387, -0.2966, -0.5836,  ..., -0.5660,  0.9143, -0.3302]]],
       grad_fn=<EmbeddingBackward0>)
tensor([[[ 0.2243,  0.0677,  1.4629,  ...,  0.9322,  0.1857,  2.4300],
         [-0.2129,  0.1500,  1.9862,  ...,  0.0000, -0.8497,  0.0727],
         [ 0.0000, -1.5058,  2.5303,  ...,  0.9322,  0.1857,  2.4300],
         [-1.3306, -0.0000, -0.2537,  ...,  0.4823,  1.0159,  0.7443]]],
       grad_fn=<MulBackward0>)


#### 总结
在使用LLM（大型语言模型，如Transformer）进行语义理解时，原始文本经过词向量化后，词语的语义信息是相同的，因为位置信息被忽略了。这可能导致模型无法正确理解文本中词语的语境和关系。
为了解决这个问题，引入了位置编码器。位置编码器的作用是为每个词向量添加位置信息，以便模型可以区分不同位置的词语，并更好地理解文本的语义。正弦和余弦函数是一种常用的方式来构建位置编码，因为它们能够生成不同位置的编码，并且这些编码可以与词向量相加，从而生成包含位置信息的文本向量。
通过将词向量和位置编码相结合，最终生成的文本向量包含了词语的语义信息和它们在句子中的位置信息。这使得模型能够更准确地捕捉文本中的语境和关系，从而提高了自然语言处理任务的性能，如机器翻译、文本生成和文本分类等。位置编码是Transformer模型中的一个关键组成部分，它有助于模型处理序列数据，特别是在处理长文本时表现出色